So you have walk-forward back-tests, which is the back tests you and I know. And then quite naturally, we have backtests on simulated data. 

I can imagine another type of backtests and that is superimposing signals from prices onto new prices to recreate new prices based on multiple scenarios. 

Another potential is to have multiple time-series and then to use back-testing as a tool to fit on - it is going to be harder to fit on multiple ones, in the sense that you do cross-validation. 

Naturally walk forward leads to higher variance, higher variance leads to more false discovery.



In [2]:
import pandas as pd
import numpy as np

def apply_func(x):
    return x ** 2

def func(df, timestamps, f):
    df_ = df.loc[timestamps]
    for idx, x in df_.items():
        df_.loc[idx] = f(x)
    return df_
    
df = pd.Series(np.random.randn(10000))
from finance_ml.multiprocessing import mp_pandas_obj

results = mp_pandas_obj(func, pd_obj=('timestamps', df.index),
                        num_threads=24, df=df, f=apply_func)
print(results.head())

0    1.084807
1    1.210773
2    0.563314
3    1.828434
4    0.533735
dtype: float64


2018-09-28 07:47:39.753567 100.0% func done after 0.01 minutes. Remaining 0.0 minutes.


In [1]:
from finance_ml.labeling import get_barrier_labels, cusum_filter
from finance_ml.stats import get_daily_vol

vol = get_daily_vol(close)
trgt = vol
timestamps = cusum_filter(close, vol)
labels = get_barrier_labels(close, timestamps, trgt, sltp=[1, 1],
                            num_days=1, min_ret=0, num_threads=16)
print(labels.show())

NameError: name 'close' is not defined

From here forward you would like to learn more about Chapter 3

In [4]:
import pandas as pd

df = pd.read_csv("datasets/Google.csv")
df.index = pd.DatetimeIndex(df['Date'].values)
close = df["Close"]

In [184]:
def getDailyVol(close,span=100):
    # daily vol, reindexed to close
    df=close.index.searchsorted(close.index-pd.Timedelta(days=1))
    df=df[df>0]
    df=pd.Series(close.index[df-1], index=close.index[close.shape[0]-df.shape[0]:])
    df=close.loc[df.index]/close.loc[df.values].values-1 # daily returns
    df=df.ewm(span=span).std()
    return df

In [33]:
import numpy as np
import pandas as pd

### This really gets a moving average of the daily volatility. 
def get_daily_vol(close, span=100):
    # The insertion index is the place to add your value to keep the index in sorted order
    use_idx = close.index.searchsorted(close.index - pd.Timedelta(days=1))
    use_idx = use_idx[use_idx > 0]
    # Get rid of duplications in index
    use_idx = np.unique(use_idx)
    prev_idx = pd.Series(close.index[use_idx - 1], index=close.index[use_idx])
    ret = close.loc[prev_idx.index] / close.loc[prev_idx.values].values - 1
    # Ewm is an exponentially weighted moving average, various parameters can be tweaked 
    vol = ret.ewm(span=span).std()
    return vol



In [74]:
### Time to simplify the formula, to write it in as little code as possible.
### This value is slightly better but the formula is better.
### I have noticed, this is the gaps, he doesn't take the returns of the
### gaps, that is fine with me, I might still wonder about the
## justification of that, but it is done. 
## basically you can ask him why he is leaving out the gap. 


import numpy as np
import pandas as pd

### This really gets a moving average of the daily volatility. 
def daily_moving_vol(close, span=100):
    if close.shape[0]==close.index.drop_duplicates().shape[0]:
        # Ewm is an exponentially weighted moving average, various parameters can be tweaked like span
        vol = ((close/close.shift(-1)-1).drop_duplicates().dropna()).ewm(span=span).std()
        return vol
    else:
        print("duplicate indexes - fix")



In [ ]:
vol = ((close/close.shift(-1)-1).drop_duplicates().dropna()).ewm(span=span).std()


In [ ]:
## It is a genius piece of code but it is a hack

In [162]:
close.head()

2004-08-19    100.335
2004-08-20    108.310
2004-08-23    109.400
2004-08-24    104.870
2004-08-25    106.000
Name: Close, dtype: float64

In [163]:
#### Interesting, my EW takes friday out.
#### His takes monday out

span = 100
days = pd.Series(close[1:].index.append(close.index.shift(1,freq="D")[-1:]) - close.index  < pd.Timedelta(days=2))
days = days.shift(1).dropna()
days = days[days==True]

ret = ((close/close.shift(-1)-1).drop_duplicates().dropna())

ew = ret.iloc[days.index].ewm(span=span).std()

In [165]:
ew.shape

(2449,)

In [172]:
close.head(15)

2004-08-19    100.335
2004-08-20    108.310
2004-08-23    109.400
2004-08-24    104.870
2004-08-25    106.000
2004-08-26    107.910
2004-08-27    106.150
2004-08-30    102.010
2004-08-31    102.370
2004-09-01    100.250
2004-09-02    101.510
2004-09-03    100.010
2004-09-07    101.580
2004-09-08    102.300
2004-09-09    102.310
Name: Close, dtype: float64

In [170]:
ew.head(15)

2004-08-20         NaN
2004-08-24    0.000493
2004-08-25    0.004299
2004-08-26    0.015255
2004-08-27    0.024609
2004-08-31    0.022998
2004-09-01    0.022231
2004-09-02    0.020847
2004-09-03    0.020728
2004-09-08    0.019472
2004-09-09    0.020882
2004-09-10    0.020700
2004-09-14    0.019728
2004-09-15    0.019321
2004-09-16    0.019889
Name: Close, dtype: float64

In [189]:
vol1 = vol1.to_frame()
vol1["date"] = list(vol1.index)

In [190]:
## Day of week`a

vol1["dow"] = vol1["date"].dt.day_name()

In [191]:
vol1

,Close,date,dow
2004-08-23,NaN,2004-08-23,Monday
2004-08-24,0.036396,2004-08-24,Tuesday
2004-08-25,0.029930,2004-08-25,Wednesday
2004-08-26,0.027366,2004-08-26,Thursday
2004-08-30,0.029365,2004-08-30,Monday
2004-08-31,0.026605,2004-08-31,Tuesday
2004-09-01,0.024794,2004-09-01,Wednesday
2004-09-02,0.024097,2004-09-02,Thursday
2004-09-07,0.023610,2004-09-07,Tuesday
2004-09-08,0.022400,2004-09-08,Wednesday


In [188]:
vol1 = get_daily_vol(df["Close"])
vol1.shape

(2472,)

In [ ]:
import numpy as np
import pandas as pd

### This really gets a moving average of the daily volatility. 
def get_daily_vol(close, span=100):
    # Easy, search the index for where the next index is a day away from previous index
    # It will return duplicate indexes, because its relative to before and after
    # Returns closest index to another. 
    # It is a hack - where index should be added to maintain order.
    use_idx = close.index.searchsorted(close.index - pd.Timedelta(days=1))
    use_idx = use_idx[use_idx > 0]
    # Get rid of duplications in index
    use_idx = np.unique(use_idx)
    prev_idx = pd.Series(close.index[use_idx - 1], index=close.index[use_idx])
    ret = close.loc[prev_idx.index] / close.loc[prev_idx.values].values - 1
    # Ewm is an exponentially weighted moving average, various parameters can be tweaked 
    vol = ret.ewm(span=span).std()
    return vol


In [62]:
close.head()

2004-08-19    100.335
2004-08-20    108.310
2004-08-23    109.400
2004-08-24    104.870
2004-08-25    106.000
Name: Close, dtype: float64

In [80]:
vol1 = get_daily_vol(df["Close"])
vol1.shape

(2472,)

In [78]:
vol = daily_moving_vol(df["Close"])
vol.shape

(3124,)

In [82]:
vol1.head(20)

2004-08-23         NaN
2004-08-24    0.036396
2004-08-25    0.029930
2004-08-26    0.027366
2004-08-30    0.029365
2004-08-31    0.026605
2004-09-01    0.024794
2004-09-02    0.024097
2004-09-07    0.023610
2004-09-08    0.022400
2004-09-09    0.021152
2004-09-13    0.021182
2004-09-14    0.022876
2004-09-15    0.021867
2004-09-16    0.021314
2004-09-20    0.020706
2004-09-21    0.020498
2004-09-22    0.019782
2004-09-23    0.019554
2004-09-27    0.019453
Name: Close, dtype: float64

In [84]:
close

2004-08-19    100.335
2004-08-20    108.310
2004-08-23    109.400
2004-08-24    104.870
2004-08-25    106.000
2004-08-26    107.910
2004-08-27    106.150
2004-08-30    102.010
2004-08-31    102.370
2004-09-01    100.250
2004-09-02    101.510
2004-09-03    100.010
2004-09-07    101.580
2004-09-08    102.300
2004-09-09    102.310
2004-09-10    105.330
2004-09-13    107.500
2004-09-14    111.490
2004-09-15    112.000
2004-09-16    113.970
2004-09-17    117.490
2004-09-20    119.360
2004-09-21    117.840
2004-09-22    118.380
2004-09-23    120.820
2004-09-24    119.830
2004-09-27    118.260
2004-09-28    126.860
2004-09-29    131.080
2004-09-30    129.600
               ...   
2016-12-02    764.460
2016-12-05    778.220
2016-12-06    776.180
2016-12-07    791.470
2016-12-08    795.170
2016-12-09    809.450
2016-12-12    807.900
2016-12-13    815.340
2016-12-14    817.890
2016-12-15    815.650
2016-12-16    809.840
2016-12-19    812.500
2016-12-20    815.200
2016-12-21    812.200
2016-12-22

In [83]:
vol.head(20)

2004-08-19         NaN
2004-08-20    0.045020
2004-08-23    0.058456
2004-08-24    0.047500
2004-08-25    0.041005
2004-08-26    0.038556
2004-08-27    0.039770
2004-08-30    0.036647
2004-08-31    0.034956
2004-09-01    0.033110
2004-09-02    0.031601
2004-09-03    0.030416
2004-09-07    0.029043
2004-09-08    0.027759
2004-09-09    0.027708
2004-09-10    0.027035
2004-09-13    0.027328
2004-09-14    0.026376
2004-09-15    0.025650
2004-09-16    0.025469
Name: Close, dtype: float64

In [7]:
close = df["Close"]

In [108]:
use_idx = close.index.searchsorted(close.index - pd.Timedelta(days=1))

In [112]:
close.head(13)

2004-08-19    100.335
2004-08-20    108.310
2004-08-23    109.400
2004-08-24    104.870
2004-08-25    106.000
2004-08-26    107.910
2004-08-27    106.150
2004-08-30    102.010
2004-08-31    102.370
2004-09-01    100.250
2004-09-02    101.510
2004-09-03    100.010
2004-09-07    101.580
Name: Close, dtype: float64

In [109]:
use_idx

array([   0,    0,    2,    2,    3,    4,    5,    7,    7,    8,    9,
         10,   12,   12,   13,   14,   16,   16,   17,   18,   19,   21,
         21,   22,   23,   24,   26,   26,   27,   28,   29,   31,   31,
         32,   33,   34,   36,   36,   37,   38,   39,   41,   41,   42,
         43,   44,   46,   46,   47,   48,   49,   51,   51,   52,   53,
         54,   56,   56,   57,   58,   59,   61,   61,   62,   63,   64,
         66,   66,   67,   69,   70,   70,   71,   72,   73,   75,   75,
         76,   77,   78,   80,   80,   81,   82,   83,   85,   85,   86,
         87,   89,   89,   90,   91,   92,   94,   94,   95,   96,   97,
         99,   99,  100,  101,  102,  104,  104,  105,  106,  108,  108,
        109,  110,  111,  113,  113,  114,  115,  116,  118,  118,  119,
        120,  121,  123,  123,  124,  125,  126,  128,  128,  129,  130,
        132,  132,  133,  134,  135,  137,  137,  138,  139,  140,  142,
        142,  143,  144,  145,  147,  147,  148,  1

In [104]:
close.index.se

AttributeError: 'DatetimeIndex' object has no attribute 'se'

In [105]:
close.index

DatetimeIndex(['2004-08-19', '2004-08-20', '2004-08-23', '2004-08-24',
               '2004-08-25', '2004-08-26', '2004-08-27', '2004-08-30',
               '2004-08-31', '2004-09-01',
               ...
               '2017-01-03', '2017-01-04', '2017-01-05', '2017-01-06',
               '2017-01-09', '2017-01-10', '2017-01-11', '2017-01-12',
               '2017-01-13', '2017-01-17'],
              dtype='datetime64[ns]', length=3125, freq=None)

In [106]:
import numpy
numpy.set_printoptions(threshold=numpy.nan)

In [107]:
use_idx


array([   0,    0,    2,    2,    3,    4,    5,    7,    7,    8,    9,
         10,   12,   12,   13,   14,   16,   16,   17,   18,   19,   21,
         21,   22,   23,   24,   26,   26,   27,   28,   29,   31,   31,
         32,   33,   34,   36,   36,   37,   38,   39,   41,   41,   42,
         43,   44,   46,   46,   47,   48,   49,   51,   51,   52,   53,
         54,   56,   56,   57,   58,   59,   61,   61,   62,   63,   64,
         66,   66,   67,   69,   70,   70,   71,   72,   73,   75,   75,
         76,   77,   78,   80,   80,   81,   82,   83,   85,   85,   86,
         87,   89,   89,   90,   91,   92,   94,   94,   95,   96,   97,
         99,   99,  100,  101,  102,  104,  104,  105,  106,  108,  108,
        109,  110,  111,  113,  113,  114,  115,  116,  118,  118,  119,
        120,  121,  123,  123,  124,  125,  126,  128,  128,  129,  130,
        132,  132,  133,  134,  135,  137,  137,  138,  139,  140,  142,
        142,  143,  144,  145,  147,  147,  148,  1

In [14]:
prev_idx

2004-08-23   2004-08-20
2004-08-24   2004-08-23
2004-08-25   2004-08-24
2004-08-26   2004-08-25
2004-08-30   2004-08-27
2004-08-31   2004-08-30
2004-09-01   2004-08-31
2004-09-02   2004-09-01
2004-09-07   2004-09-03
2004-09-08   2004-09-07
2004-09-09   2004-09-08
2004-09-13   2004-09-10
2004-09-14   2004-09-13
2004-09-15   2004-09-14
2004-09-16   2004-09-15
2004-09-20   2004-09-17
2004-09-21   2004-09-20
2004-09-22   2004-09-21
2004-09-23   2004-09-22
2004-09-27   2004-09-24
2004-09-28   2004-09-27
2004-09-29   2004-09-28
2004-09-30   2004-09-29
2004-10-04   2004-10-01
2004-10-05   2004-10-04
2004-10-06   2004-10-05
2004-10-07   2004-10-06
2004-10-11   2004-10-08
2004-10-12   2004-10-11
2004-10-13   2004-10-12
                ...    
2016-11-21   2016-11-18
2016-11-22   2016-11-21
2016-11-25   2016-11-23
2016-11-28   2016-11-25
2016-11-29   2016-11-28
2016-11-30   2016-11-29
2016-12-01   2016-11-30
2016-12-05   2016-12-02
2016-12-06   2016-12-05
2016-12-07   2016-12-06
2016-12-08   201

In [10]:
use_idx = use_idx[use_idx > 0]
# Get rid of duplications in index
use_idx = np.unique(use_idx)

In [13]:
prev_idx = pd.Series(close.index[use_idx - 1], index=close.index[use_idx])

In [15]:
ret = close.loc[prev_idx.index] / close.loc[prev_idx.values].values - 1

In [ ]:
vol = ret.ewm(span=span).std()

In [18]:
vol = get_daily_vol(df["Close"])
vol.head()

2004-08-23         NaN
2004-08-24    0.036396
2004-08-25    0.029930
2004-08-26    0.027366
2004-08-30    0.029365
Name: Close, dtype: float64

In [5]:
close

2004-08-19    100.335
2004-08-20    108.310
2004-08-23    109.400
2004-08-24    104.870
2004-08-25    106.000
2004-08-26    107.910
2004-08-27    106.150
2004-08-30    102.010
2004-08-31    102.370
2004-09-01    100.250
2004-09-02    101.510
2004-09-03    100.010
2004-09-07    101.580
2004-09-08    102.300
2004-09-09    102.310
2004-09-10    105.330
2004-09-13    107.500
2004-09-14    111.490
2004-09-15    112.000
2004-09-16    113.970
2004-09-17    117.490
2004-09-20    119.360
2004-09-21    117.840
2004-09-22    118.380
2004-09-23    120.820
2004-09-24    119.830
2004-09-27    118.260
2004-09-28    126.860
2004-09-29    131.080
2004-09-30    129.600
               ...   
2016-12-02    764.460
2016-12-05    778.220
2016-12-06    776.180
2016-12-07    791.470
2016-12-08    795.170
2016-12-09    809.450
2016-12-12    807.900
2016-12-13    815.340
2016-12-14    817.890
2016-12-15    815.650
2016-12-16    809.840
2016-12-19    812.500
2016-12-20    815.200
2016-12-21    812.200
2016-12-22

In [192]:
import numbers

def cusum_filter(close, h):
    # asssum that E y_t = y_{t-1}
    t_events = []
    s_pos, s_neg = 0, 0
    ret = close.pct_change().dropna()
    diff = ret.diff().dropna()
    # time variant threshold
    if isinstance(h, numbers.Number):
        h = pd.Series(h, index=diff.index)
    h = h.reindex(diff.index, method='bfill')
    h = h.dropna()
    for t in h.index:
        s_pos = max(0, s_pos + diff.loc[t])
        s_neg = min(0, s_neg + diff.loc[t])
        if s_pos > h.loc[t]:
            s_pos = 0
            t_events.append(t)
        elif s_neg < -h.loc[t]:
            s_neg = 0
            t_events.append(t)
    return pd.DatetimeIndex(t_events)

In [193]:
cusum_filter(df["Close"], 0.1)

DatetimeIndex(['2004-08-24', '2004-09-28', '2004-10-20', '2004-10-21',
               '2004-10-26', '2004-11-05', '2004-11-08', '2004-11-11',
               '2004-11-16', '2004-11-24', '2005-01-26', '2005-02-04',
               '2005-06-08', '2005-10-21', '2005-10-25', '2006-01-20',
               '2006-01-23', '2006-02-01', '2006-02-16', '2006-02-28',
               '2006-03-02', '2006-03-24', '2006-04-25', '2006-10-20',
               '2006-11-27', '2008-01-23', '2008-01-24', '2008-02-01',
               '2008-02-05', '2008-03-11', '2008-03-17', '2008-03-24',
               '2008-04-18', '2008-04-21', '2008-07-18', '2008-07-22',
               '2008-09-11', '2008-09-17', '2008-09-18', '2008-09-22',
               '2008-09-29', '2008-09-30', '2008-10-02', '2008-10-13',
               '2008-10-14', '2008-10-16', '2008-10-28', '2008-10-29',
               '2008-11-05', '2008-11-13', '2008-11-17', '2008-11-25',
               '2008-12-01', '2008-12-02', '2008-12-22', '2009-01-20',
      